In [9]:
import os

out_dir = "data/debug_lm478"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [10]:
import numpy as np
from vfhq_dl.video_util import get_all_frames_from_video
import cv2
from PIL import Image
from face_landmarking.face_landmarker import index_lm68_from_lm478
import imageio
from tqdm import trange

def draw_lm68(img, lm68, color=None):
    color = color or (0, 255, 255)
    for i in range(len(lm68)):
        x, y = map(int, lm68[i])
        img = cv2.circle(img, (x, y), 2, color, -1)
        img = cv2.putText(img, f"{i}", org=(x,y), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=color)
    return img

def render_video_for_extracted_lm478(video_name):
    lms = np.load(f'data/video_lm478s/{video_name}.npz')
    
    lm478s = lms['lm478s']
    video_width, video_height = lms['vid_dims']
    frames = get_all_frames_from_video(f'data/cropped_videos/{video_name}.mp4', video_width, video_height)
  
    writer = imageio.get_writer(os.path.join(out_dir, video_name + ".mp4"), fps = 25, format='FFMPEG', codec='h264')

    for i in trange(len(frames)):
        lm68_drawn = draw_lm68(frames[i], lm478s[i, index_lm68_from_lm478, :])
        writer.append_data(lm68_drawn)

    writer.close()

In [11]:
from tqdm import tqdm

vid_names = [
    "Clip+z3gXzESgDsw+P0+C1",
    "Clip+5FCk2skYPAo+P0+C0",
    "Clip+L4hdt_JdfXE+P0+C0",
    "Clip+S1jNBUr_nis+P0+C0",
    "Clip+yAeWp8XNgz8+P0+C2"
]

for vname in tqdm(vid_names):
    render_video_for_extracted_lm478(vname)

100%|██████████| 5/5 [00:43<00:00,  8.61s/it]
